## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
new_column_order = ["City","Country","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Description"]
city_data_df = city_data_df[new_column_order]
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Obo,CF,5.3998,26.4869,71.67,27,100,3.76,overcast clouds
1,Yellowknife,CA,62.4560,-114.3525,-24.45,80,34,4.00,scattered clouds
2,Lodja,CD,-3.4833,23.4333,68.43,99,100,1.30,overcast clouds
3,Albany,US,42.6001,-73.9662,41.47,77,100,3.38,overcast clouds
4,Savannah Bight,HN,16.4500,-85.8500,79.00,82,0,28.30,clear sky
...,...,...,...,...,...,...,...,...,...
665,Arsk,RU,56.0915,49.8778,24.58,98,100,1.57,snow
666,Camacupa,AO,-12.0167,17.4833,61.92,93,86,1.99,overcast clouds
667,Lubango,AO,-14.9172,13.4925,58.71,53,10,5.23,clear sky
668,Barra Do Garcas,BR,-15.8900,-52.2567,73.53,95,100,3.27,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()

In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().sum().sum()

0

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Obo,CF,71.67,overcast clouds,5.3998,26.4869,
4,Savannah Bight,HN,79.00,clear sky,16.4500,-85.8500,
5,Atuona,PF,78.24,light rain,-9.8000,-139.0333,
7,Busselton,AU,76.05,broken clouds,-33.6500,115.3333,
9,Port Elizabeth,ZA,71.10,overcast clouds,-33.9180,25.5701,
18,Padang,ID,79.83,broken clouds,-0.9492,100.3543,
19,Kapaa,US,81.48,clear sky,22.0752,-159.3190,
24,Arraial Do Cabo,BR,75.25,overcast clouds,-22.9661,-42.0278,
25,Rikitea,PF,78.60,light rain,-23.1203,-134.9692,
30,Necochea,AR,72.03,moderate rain,-38.5473,-58.7368,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# 7. Drop the rows where there is no Hotel Name.
placeholder = float("NaN")
hotel_df.replace("",placeholder, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Savannah Bight,HN,79.00,clear sky,16.4500,-85.8500,Villa on Dunbar Rock
5,Atuona,PF,78.24,light rain,-9.8000,-139.0333,Villa Enata
7,Busselton,AU,76.05,broken clouds,-33.6500,115.3333,Observatory Guest House
9,Port Elizabeth,ZA,71.10,overcast clouds,-33.9180,25.5701,39 On Nile Guest House
18,Padang,ID,79.83,broken clouds,-0.9492,100.3543,Mercure Padang
...,...,...,...,...,...,...,...
656,Itarema,BR,79.18,overcast clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
658,Porto Novo,BJ,78.62,clear sky,6.4965,2.6036,Tour Eiffel Hotel Benin
663,Alice Town,BS,73.26,clear sky,25.7167,-79.3000,Bimini Big Game Club Resort & Marina
664,Anori,BR,74.64,overcast clouds,-3.7728,-61.6442,Novo Hotel 5 Irmãos


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))